In [1]:
import requests
import json
import pandas as pd
import time

In [2]:
cities_df =  pd.read_csv('collected_data/cities_list.csv')

In [3]:
header = {
    'X-ClientId': 'c7b4da28-cadf-4dd6-f3d1-08dc7b3a73a3'
}

In [4]:
gus_url = "https://bdl.stat.gov.pl/api/v1"

In [5]:
url_1 = gus_url+'/Variables?subject-id=P3583'

In [9]:
resp_1 = requests.get(url_1, headers = header)

In [11]:
id_dict = {'benzyna': None, 'olej napędowy': None, 'gaz (LPG)': None, 'pozostałe': None}

In [13]:
if resp_1.status_code == 200:
    for elem in resp_1.json()['results']:
        if elem['n1'] == 'samochody osobowe':
            if elem['n2'] == 'benzyna':
                id_dict['benzyna'] = elem['id']
            elif elem['n2'] == 'olej napędowy':
                id_dict['olej napędowy'] = elem['id']
            elif elem['n2'] == 'gaz (LPG)':
                id_dict['gaz (LPG)'] = elem['id']
            else:
                id_dict['pozostałe'] = elem['id']

In [15]:
print(id_dict)

{'benzyna': 475618, 'olej napędowy': 475614, 'gaz (LPG)': 475610, 'pozostałe': 475622}


In [59]:
df_dict = {}
for key in id_dict.keys():
    df_dict[key] = pd.DataFrame(columns = ['city', 'date', 'no_of_cars'])

In [60]:
def get_cars_data(city_name, cars_data_df, car_id):
    url_2 = gus_url+'/Units/search'
    parameters = {'level': 5, 'name': city_name}
    resp_2 = requests.get(url_2, params = parameters, headers = header)
    if resp_2.status_code == 200:
        resp_2_res = resp_2.json()['results']
        resp_id = None
        for xk in resp_2_res:
            if 'Powiat m. ' in xk['name']:
                resp_id = xk['parentId']
                break
        if resp_id:
            parameters_2 = {'unit-parent-id':resp_id}
            url_4 = f"{gus_url}/data/by-variable/{car_id}"
            resp_3 = requests.get(url_4, params=parameters_2, headers = header)
            if resp_3.status_code == 200:
                resp_3_res = resp_3.json()['results']
                for xp in resp_3_res:
                    if 'Powiat m. ' in xp['name']:
                        if 'values' in xp:
                            for val in xp['values']:
                                cars_data_df.loc[len(cars_data_df)] = [city_name, int(val['year']), int(val['val'])]
                        break
            else:
                print('\tNie udało się pobrać danych.')
    else:
        print('\tNie udało się pobrać danych.')

In [61]:
for index, row in cities_df.iterrows():
    print(f'Przetwarzanie miasta: {index+1}')
    try:
        for car_key, car_id in id_dict.items():
            get_cars_data(row['city'], df_dict[car_key], car_id)
    except Exception as e:
        print(f'\tBłąd: {e}')
    time.sleep(1)
    if (index+1)%20 == 0:
        time.sleep(60)  

Przetwarzanie miasta: 1
Przetwarzanie miasta: 2
Przetwarzanie miasta: 3
Przetwarzanie miasta: 4
Przetwarzanie miasta: 5
Przetwarzanie miasta: 6
Przetwarzanie miasta: 7
Przetwarzanie miasta: 8
Przetwarzanie miasta: 9
Przetwarzanie miasta: 10
Przetwarzanie miasta: 11
Przetwarzanie miasta: 12
Przetwarzanie miasta: 13
Przetwarzanie miasta: 14
Przetwarzanie miasta: 15
Przetwarzanie miasta: 16
Przetwarzanie miasta: 17
Przetwarzanie miasta: 18
Przetwarzanie miasta: 19
Przetwarzanie miasta: 20
Przetwarzanie miasta: 21
Przetwarzanie miasta: 22
Przetwarzanie miasta: 23
Przetwarzanie miasta: 24
Przetwarzanie miasta: 25
Przetwarzanie miasta: 26
Przetwarzanie miasta: 27
Przetwarzanie miasta: 28
Przetwarzanie miasta: 29
Przetwarzanie miasta: 30
Przetwarzanie miasta: 31
Przetwarzanie miasta: 32
Przetwarzanie miasta: 33
Przetwarzanie miasta: 34
Przetwarzanie miasta: 35
Przetwarzanie miasta: 36
Przetwarzanie miasta: 37
Przetwarzanie miasta: 38
Przetwarzanie miasta: 39
Przetwarzanie miasta: 40
Przetwarz

In [63]:
for df_dict_key, df_dict_value in df_dict.items():
    df_dict_value.to_csv(f'collected_data/cars_number_{df_dict_key}.csv', index=False)

In [64]:
url_11 = gus_url+'/Variables?subject-id=P3584'

In [65]:
resp_11 = requests.get(url_11, headers = header)

In [68]:
id_dict_2 = {}

In [69]:
if resp_11.status_code == 200:
    for elem in resp_11.json()['results']:
        id_dict_2[elem['n2']] = elem['id']

In [70]:
print(id_dict_2)

{'ogółem': 475696, 'do 1 roku': 475690, '2 lata': 475684, '3 lata': 475678, '4-5 lat': 475672, '6-7 lat': 475666, '8-9 lat': 475660, '10-11 lat': 475654, '12-15 lat': 475648, '16-20 lat': 475642}


In [100]:
df_cars_ages = pd.DataFrame(columns = ['city', 'date']+[x for x in id_dict_2.keys()])

In [101]:
def get_cars_data_2(city_name, cars_data_df):
    url_2 = gus_url+'/Units/search'
    parameters = {'level': 5, 'name': city_name}
    resp_2 = requests.get(url_2, params = parameters, headers = header)
    if resp_2.status_code == 200:
        res_dict = {}
        resp_2_res = resp_2.json()['results']
        resp_id = None
        for xk in resp_2_res:
            if 'Powiat m. ' in xk['name']:
                resp_id = xk['parentId']
                break
        if resp_id:
            parameters_2 = {'unit-parent-id':resp_id}
            for car_key, car_id in id_dict_2.items():
                url_4 = f"{gus_url}/data/by-variable/{car_id}"
                resp_3 = requests.get(url_4, params=parameters_2, headers = header)
                if resp_3.status_code == 200:
                    resp_3_res = resp_3.json()['results']
                    for xp in resp_3_res:
                        if 'Powiat m. ' in xp['name']:
                            if 'values' in xp:
                                for val in xp['values']:                                    
                                    if int(val['year']) in res_dict:
                                        if len(res_dict[int(val['year'])]) == 0:
                                            res_dict[int(val['year'])] = [int(val['val'])]
                                        else:
                                            res_dict[int(val['year'])].append(int(val['val']))
                                    else:
                                         res_dict[int(val['year'])] = [int(val['val'])]
                            break
                else:
                    print('\tNie udało się pobrać danych.')
        if len(res_dict) > 0:
            for res_key, res_val in res_dict.items():
                cars_data_df.loc[len(cars_data_df)] = [city_name, res_key]+res_val
    else:
        print('\tNie udało się pobrać danych.')

In [102]:
for index, row in cities_df.iterrows():
    print(f'Przetwarzanie miasta: {index+1}')
    try:
        get_cars_data_2(row['city'], df_cars_ages)
    except Exception as e:
        print(f'\tBłąd: {e}')
    time.sleep(1)
    if (index+1)%20 == 0:
        time.sleep(60)  

Przetwarzanie miasta: 1
Przetwarzanie miasta: 2
Przetwarzanie miasta: 3
Przetwarzanie miasta: 4
Przetwarzanie miasta: 5
Przetwarzanie miasta: 6
Przetwarzanie miasta: 7
Przetwarzanie miasta: 8
Przetwarzanie miasta: 9
Przetwarzanie miasta: 10
Przetwarzanie miasta: 11
Przetwarzanie miasta: 12
Przetwarzanie miasta: 13
Przetwarzanie miasta: 14
Przetwarzanie miasta: 15
Przetwarzanie miasta: 16
Przetwarzanie miasta: 17
Przetwarzanie miasta: 18
Przetwarzanie miasta: 19
Przetwarzanie miasta: 20
Przetwarzanie miasta: 21
Przetwarzanie miasta: 22
Przetwarzanie miasta: 23
Przetwarzanie miasta: 24
Przetwarzanie miasta: 25
Przetwarzanie miasta: 26
Przetwarzanie miasta: 27
Przetwarzanie miasta: 28
Przetwarzanie miasta: 29
Przetwarzanie miasta: 30
Przetwarzanie miasta: 31
Przetwarzanie miasta: 32
Przetwarzanie miasta: 33
Przetwarzanie miasta: 34
Przetwarzanie miasta: 35
Przetwarzanie miasta: 36
Przetwarzanie miasta: 37
Przetwarzanie miasta: 38
Przetwarzanie miasta: 39
Przetwarzanie miasta: 40
Przetwarz

In [105]:
df_cars_ages.info()

<class 'pandas.core.frame.DataFrame'>
Index: 384 entries, 0 to 383
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   city       384 non-null    object
 1   date       384 non-null    int64 
 2   ogółem     384 non-null    int64 
 3   do 1 roku  384 non-null    int64 
 4   2 lata     384 non-null    int64 
 5   3 lata     384 non-null    int64 
 6   4-5 lat    384 non-null    int64 
 7   6-7 lat    384 non-null    int64 
 8   8-9 lat    384 non-null    int64 
 9   10-11 lat  384 non-null    int64 
 10  12-15 lat  384 non-null    int64 
 11  16-20 lat  384 non-null    int64 
dtypes: int64(11), object(1)
memory usage: 39.0+ KB


In [106]:
df_cars_ages['city'].nunique()

48

In [107]:
df_cars_ages.to_csv('collected_data/cars_number_age_depend.csv', index=False)